## 🌟 LLM finetuning 해보기

- 본 프로젝트는 증강한 데이터를 바탕으로 LLM(Qwen3-1.7b)을 파인튜닝해보는 프로젝트입니다.
- 해당 프로젝트는 코랩 T4 환경을 고려하여 모델 학습을 진행했습니다.
- 즉 무료 코랩 버전으로도 파인튜닝 실습이 가능합니다.

### 데이터 전처리

In [ ]:
# 필요한 라이브러리 설치
!pip install -q transformers==4.56.1
!pip install -q datasets==4.0.0
!pip install -q accelerate==1.10.1
!pip install -q trl==0.22.2
!pip install -q peft==0.17.1

In [ ]:
# gpu 사용 가능한지 확인
import torch

if torch.cuda.is_available():
    print(f"사용 중인 GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU 사용 불가능")

In [ ]:
# 허깅페이스 모델 다운을 위한 허깅페이스 로그인
import huggingface_hub
from google.colab import userdata

huggingface_hub.login(userdata.get('huggingface_api_key'))

In [ ]:
# 허깅페이스에서 데이터셋 내려 받아서 파인튜닝을 위한 데이터 준비하기
from datasets import load_dataset

# 허깅페이스 허브에서 데이터셋 로드
dataset_train = load_dataset("kkobuking/finetuning_test_data_0907", split="train")
dataset_valid = load_dataset("kkobuking/finetuning_test_data_0907", split="validation")
dataset_test = load_dataset("kkobuking/finetuning_test_data_0907", split="test")

# 원본 데이터의 type별 분포 출력
# 데이터셋에 type 열이 없으므로 전체 데이터 크기만 출력
print("훈련 데이터 크기:", len(dataset_train), "/ 검증 데이터 크기", len(dataset_valid), "/ 테스트 데이터 크기", len(dataset_test))


# OpenAI 형식으로 데이터 변환을 위한 함수
def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": sample["system"],
            },
            {
                "role": "user",
                "content": sample["user"],
            },
            {
                "role": "assistant",
                "content": str(sample["assistant"])
            },
        ],
    }

# 7. 분할된 데이터를 OpenAI format으로 변환
train_dataset = [format_data(dataset_train[i]) for i in range(len(dataset_train))]
valid_dataset = [format_data(dataset_valid[i]) for i in range(len(dataset_valid))]
test_dataset = [format_data(dataset_test[i]) for i in range(len(dataset_test))]

# 8. 최종 데이터셋 크기 출력
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Valid {len(valid_dataset)}개, Test {len(test_dataset)}개")

In [ ]:
# 데이터 확인해보기
train_dataset[0]["messages"]

In [ ]:
# 리스트 형태에서 다시 Dataset 객체로 변경
from datasets import Dataset

train_dataset = Dataset.from_list(train_dataset)
valid_dataset = Dataset.from_list(valid_dataset)
test_dataset = Dataset.from_list(test_dataset)

In [ ]:
# Dataset 형태 확인해보기
print(train_dataset[0]["messages"][1]["content"])

### 모델 로드 및 템플릿 적용 테스트

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 허깅페이스 모델 ID
model_id = "Qwen/Qwen3-1.7B" #GPU 환경에 맞게 모델을 선택할 수 있음

# 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # 허깅페이스 모델 ID
# model_id = "Qwen/Qwen3-1.7B" #GPU 환경에 맞게 모델을 선택할 수 있음

# # Load tokenizer first
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# # Qwen 모델의 경우 특수 토큰 설정
# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.im_end  # Qwen에서는 보통 im_end를 패딩으로 사용

# # Load model
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
# )

# # 모델 설정을 토크나이저와 일치시키기
# model.config.pad_token_id = tokenizer.pad_token_id
# if hasattr(model, 'generation_config') and model.generation_config is not None:
#     model.generation_config.pad_token_id = tokenizer.pad_token_id

# # 확인차 출력
# print(f"Tokenizer pad_token: {tokenizer.pad_token}")
# print(f"Tokenizer pad_token_id: {tokenizer.pad_token_id}")
# print(f"Model config pad_token_id: {model.config.pad_token_id}")

In [ ]:
# 데이터에 다운 받은 모델의 템플릿 적용해보기
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

### LoRA와 SFTConfig 설정

In [ ]:
# LoRA 파인튜닝을 위한 설정
from peft import LoraConfig

peft_config = LoraConfig(
        lora_alpha=16, # 더해지는 정도를 결정하는 계수
        lora_dropout=0.1, # 드롭아웃 확률
        r=4, # 학습시키는 행렬의 크기를 결정하는 계수
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

In [ ]:
from trl import SFTConfig, SFTTrainer

# 최대 길이
max_seq_length=2048

args = SFTConfig(
    output_dir="qwen3-1.7b-test",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수
    per_device_train_batch_size=2,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 일단은 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to="none"
)

### 학습 중 전처리 함수

In [ ]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)

        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()

        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=2048, # 여기서 max_seq_length 지정
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        # 레이블 초기화
        labels = [-100] * len(input_ids)

        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"

        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)

        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):

                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):

                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)

                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1

                    i = current_pos

            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])

    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])

        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)

    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)

    return new_batch

In [ ]:
# collate_fn 테스트 (배치 크기 1로)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)

In [ ]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

In [ ]:
print('어텐션 마스크에 대한 인코딩 결과:')
print(batch["attention_mask"][0].tolist())

In [ ]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

### 모델 학습 및 저장

In [ ]:
args.label_names = ["labels"]

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

In [ ]:
# 학습 시작
trainer.train()   # 모델이 자동으로 output_dir에 저장됨

In [ ]:
# 어댑터 모델과 토크나이저 모두 저장
output_dir = "./qwen3-1.7b-finetuning_test-lora-adapter"  # 원하는 경로

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"모델이 {output_dir}에 저장되었습니다.")

# 모델 허깅페이스 업로드
trainer.model.push_to_hub("kkobuking/qwen3-1.7b-finetuning_test-lora-adapter", private=False)
tokenizer.push_to_hub("kkobuking/qwen3-1.7b-finetuning_test-lora-adapter", private=False)

print(f"모델이 허깅페이스에 저장되었습니다.")

In [ ]:
# 만약 삭제하고 싶다면
from huggingface_hub import delete_repo

# 허깅페이스에서 삭제
if False:
  delete_repo("kkobuking/qwen3-1.7b-finetuning_test-lora-adapter")

# 로컬 삭제
import shutil

if False:
  shutil.rmtree("./qwen3-1.7b-finetuning_test-lora-adapter")

In [ ]:
!ls -alh qwen3-1.7b-finetuning_test-lora-adapter

In [ ]:
# checkpoint 별 모델 저장을 해놓으면 checkpoint 당시 저장된 모델을 사용 가능
!ls -alh qwen3-1.7b-test/checkpoint-60

In [ ]:
# 어댑터 모델을 다시 병합해보기 (gpu 문제 시, 여기서부터 세션을 다시 시작할 수 있음)
from peft import LoraConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# # 허깅페이스 모델 ID
# model_id = "Qwen/Qwen3-1.7B"

# # Load model and tokenizer
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
# )
# tokenizer = AutoTokenizer.from_pretrained(model_id)


# 이미 로드해둔 베이스 모델 model 위에, LoRA adapter 가중치가 저장된 디렉토리를 불러와 PEFT 모델로 wrapping
model = PeftModel.from_pretrained(model, "./qwen3-1.7b-finetuning_test-lora-adapter", device_map='auto', torch_dtype=torch.float16)

# 현재 LoRA 어댑터가 분리된 형태로 존재하는 PEFT 모델에서 adapter 파라미터를 본래 모델의 가중치에 병합, LoRA 관련 파라미터들을 메모리에서 언로드(unload)하여 일반적인 Hugging Face base model 형태로 변환
merged_model = model.merge_and_unload()
merged_model.save_pretrained('qwen3-1.7b-finetuning_test')
tokenizer.save_pretrained("qwen3-1.7b-finetuning_test")

In [ ]:
# 병합된 모델 허깅페이스 업로드
if True:
    merged_model.push_to_hub(
        "kkobuking/qwen3-1.7b-finetuning_test",
        tokenizer=tokenizer,
        private=False
    )
    tokenizer.push_to_hub("kkobuking/qwen3-1.7b-finetuning_test", private=False)

# 만약 삭제하고 싶다면
from huggingface_hub import delete_repo

if False:
  delete_repo("kkobuking/qwen3-1.7b-finetuning_test")

In [ ]:
!ls -alh qwen3-1.7b-finetuning_test

### 파인 튜닝 모델 테스트
- (중요) GPU RAM 사용 초기화를 위해, 세션을 다시 실행할 수 있음
- 다시 실행 시, 이전 '데이터 전처리'만 다시 실행

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

다음 2가지 방법 중 하나를 사용할 수 있습니다.

In [ ]:
# 병합한 모델을 불러와서 테스트해보는 방법 1
FINETUNE_MODEL = "kkobuking/qwen3-1.7b-finetuning_test"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(FINETUNE_MODEL)
pipe = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer)

In [ ]:
# 자동으로 파인튜닝했던 베이스 모델을 찾아서 불러와지도록 실행해보는 방법 2
peft_model_id = "kkobuking/qwen3-1.7b-finetuning_test-lora-adapter"
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

In [ ]:
# 테스트 데이터 준비
## 실제 모델에 입력을 넣을 때에는 입력의 뒤에 '<|im_start|>assistant' 추가하여 답변 생성 유도
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [ ]:
print(prompt_lst[0])

In [ ]:
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

In [ ]:
# 답변 바로 텍스트로 받아볼 수 있는 함수
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, temperature=0.2)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [ ]:
for prompt, label in zip(prompt_lst[0:5], label_lst[0:5]):
    # 파인튜닝된 모델이 생성한 답변
    print(f" 파인튜닝 모델 :\n{test_inference(pipe, prompt)}")
    # 기존에 데이터에 있던 답변
    print(f" 데이터 라벨:\n{label}")
    print("-"*50)

In [ ]:
# 직접 입력하고 싶은 내용 입력해서 답변 받아보기
messages = [
    {"role": "system", "content": '''You are a test item developer who generates English passages based on given Korean news articles.
You must create an English passage that is grounded in academic content, deals with fair and objective facts, and reflects the specified field and topic provided by the user.
The passage should maintain a coherent flow around a single theme,
without unnecessary repetition of the same information or similar arguments, while developing each sentence logically.
Rather than merely listing information, you should connect concepts organically and describe them logically.
The passage must be written in such a way that students can engage in logical reasoning while reading it.
All sentences must be generated in English and be grammatically flawless.'''},
    {"role": "user", "content": '''Please create an English passage
Field : Technology
Topic : Iron Man Suit is going to be sold about future'''}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print(test_inference(pipe, text))